# Collect articles about hurricane Helene coverage

1. Use wayback Machine to scrape everything from a news website

2. Use UNC access's News Bank

## 2. News Bank pdf about "Helene" coverage

Use Selenium to download news article pdfs from news bank

keyword: helene

date: Sep 2024 (when the first 3 articles about Helene were published) - Oct 2024?

Location: USA - North Carolina

In [ ]:
# Import Module
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Scrape multiple pages

In [ ]:
# Open Chrome
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

# Open URL
url = "https://infoweb.newsbank.com/apps/news/results?hide_duplicates=2&offset=0&maxresults=60&sort=YMD_date%3AD&p=AMNEWS&t=country%3AUSA%21USA/state%3ANC%21USA%2B-%2BNorth%2BCarolina&f=advanced&val-base-0=helene&fld-base-0=alltext&bln-base-1=and&val-base-1=Sep%202024%20-%20Oct%202024&fld-base-1=YMD_date"
driver.get(url)
time.sleep(5)
# Download pdf
try:
    # For I pages
    for i in range(2):
        # Click "select all" box
        # await driver.find_element(By.CLASS_NAME, 'search-hits__select-all form-checkbox').click()
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "search-hits__select-all.form-checkbox"))).click()
        # Click next
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "pager__item.pager__item--next"))).click()
        time.sleep(1)
except Exception as e:
    print(f"Cannot select page {i}")
    print(e)
else:
    # Click download button
    driver.find_element(By.CLASS_NAME, 'actions-bar__button.actions-bar__button--download').click()
    time.sleep(1)
    # While there is "Next" button
    while True:
        # Click "Download" button in pop-up
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "u-button multidoc-download__button"))).click()
        # wait loader to appear and then disappear
        WebDriverWait(driver, 10).until(EC.element_to_be_selected((By.CLASS_NAME, 'multidoc__button-spinner')))
        WebDriverWait(driver, 10).until(EC.invisibility_of_element((By.CLASS_NAME, 'multidoc__button-spinner')))
        # Click "Next"
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "multidocs_pager_nav next"))).click()
        except:
            print("No Next found")
            break

driver.quit()





There is security check of robot. Can't pass ?

Ended up with manually downloading 5000+ articles by groups

In [1]:
import pdfplumber
import os
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re


extract text from pdf with <a href = 'https://github.com/jsvine/pdfplumber'>pdfplumber</a>

In [3]:
#   FUNC extract_text_from_pdf()
#   Input: directory (str) of pdf
#   Output: header (list) - article header information
#         article (list) - article text
def extract_text_from_pdf(dir, headers, articles):
    # header, article = [],[]
    with pdfplumber.open(dir) as pdf:
        for page in pdf.pages:
            # extract texts on each page and split texts by "OpenURL Link"
            texts = page.extract_text().split("\nOpenURL Link\n")
            # there is no "OpenURL Link" on that page
            if len(texts) == 1:
                articles[-1] += texts[0]
            else:
                headers.append(texts[0])
                articles.append(texts[1])
    return 

In [14]:
#   FUNC: decompose_header
#   Input: header (str)
#   Output: append titles, dates, newspapers, authors, and word_counts
def decompose_header(header):


    date_match = re.search(r"\b(September|October) \d{1,2}, \d{4}\b", header)
    
    date = date_match.group(0) if date_match else ""
    dates.append(date)
    loc = header.find(date)
    titles.append(header[:loc].replace("\n", ""))

    texts = header[loc:].split('\n')
    newspapers.append(texts[0][len(date)+1:].replace('| ', ""))

    # Line three
    match = re.search(r"Author: (.*?)Section: ", texts[1])
    author = match.group(1) if match else ""
    authors.append(author)
    match_word = re.search(r"(\d+)\s*Words", texts[1])
    word_count = match_word.group(1) if match_word else ""
    word_counts.append(word_count)

    return

In [ ]:
# Get all files under news_bank_pdf
directory = 'news_bank_pdf'
pdf_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".pdf")]

headers, articles = [],[]
# Extract articles and headers from pdfs
for path in pdf_paths:
    extract_text_from_pdf(path, headers, articles)

# Save articles and headers to dataframe
temp = pd.DataFrame({"header":headers,
                    "article": articles})

# temp.to_csv('articles_20241112.csv', index=False)

In [ ]:
# Decompose headers
titles, dates, newspapers, authors, word_counts = [],[],[],[],[]
temp['header'].apply(lambda x: decompose_header(x))

temp['title'] = titles
temp['date'] = dates
temp['newspaper'] = newspapers
temp['author'] = authors
temp['word_count'] = word_counts
temp.head()

# temp.to_csv("articles_20241112_cleaned.csv", index=False)

In [ ]:
def extract_text_tru_link(url):
    # Open Chrome
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)

    # Open URL
    # url = "https://infoweb.newsbank.com/apps/news/results?hide_duplicates=2&offset=0&maxresults=60&sort=YMD_date%3AD&p=AMNEWS&t=country%3AUSA%21USA/state%3ANC%21USA%2B-%2BNorth%2BCarolina&f=advanced&val-base-0=helene&fld-base-0=alltext&bln-base-1=and&val-base-1=Sep%202024%20-%20Oct%202024&fld-base-1=YMD_date"
    driver.get(url)
    time.sleep(5)

    # WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "search-hits__select-all.form-checkbox"))).click()
    driver.quit()
    return

extract_text_tru_link('https://infoweb.newsbank.com/apps/news/user/login?destination=results%3Fhide_duplicates%3D2%26offset%3D0%26maxresults%3D60%26sort%3DYMD_date%253AD%26p%3DAMNEWS%26t%3Dcountry%253AUSA%2521USA/state%253ANC%2521USA%252B-%252BNorth%252BCarolina%26f%3Dadvanced%26val-base-0%3Dhelene%26fld-base-0%3Dalltext%26bln-base-1%3Dand%26val-base-1%3DSep%25202024%2520-%2520Oct%25202024%26fld-base-1%3DYMD_date')


    

### Extract text from newspaper layout with pdfplumber

In [ ]:
import pdfplumber

with pdfplumber.open("News_Article__Fayetteville_Observer_The_NC___October_8_2024__p1.pdf") as pdf:
    for page in pdf.pages:
        text = page.extract_text(x_tolerance=2, y_tolerance=2)  # Adjust for layout
        print(text)


### Extract text from newspaper layout with PyMuPDF

In [ ]:
import fitz  # PyMuPDF

# Open the PDF file
pdf = fitz.open("News_Article__Fayetteville_Observer_The_NC___October_8_2024__p1.pdf")

for page_num in range(pdf.page_count):
    page = pdf[page_num]
    blocks = page.get_text("blocks")  # Extract text blocks with coordinates

    # Sort blocks by y-coordinate for columnar layout
    blocks.sort(key=lambda block: (block[1], block[0]))  # y, x sorting
    for block in blocks:
        print(block[4])  # Text in each block

### Extract text from newspaper layout with Tesseract

Install <a href='https://github.com/tesseract-ocr/tesseract'>Tesseract</a> through homebrew

In [ ]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image

In [ ]:
# Convert PDF pages to images
pages = convert_from_path("News_Article__Fayetteville_Observer_The_NC___October_8_2024__p1.pdf")

for page in pages:
    # Crop or split image into columns if necessary
    text = pytesseract.image_to_string(page, config='--psm 6')  # PSM 6 for uniform block detection
    print(text)


### Extract text from newspaper layout with Layout parser

In [ ]:
import layoutparser as lp
from layoutparser.models.detectron2 import catalog
import copy
import os
import requests as requests
import layoutparser as lp
from pdf2image import convert_from_path

def load_model(
        config_path: str = 'lp://<dataset_name>/<model_name>/config',
):

    config_path_split = config_path.split('/')
    dataset_name = config_path_split[-3]
    model_name = config_path_split[-2]

    # get the URLs from the MODEL_CATALOG and the CONFIG_CATALOG 
    # (global variables .../layoutparser/models/detectron2/catalog.py)
    model_url = catalog.MODEL_CATALOG[dataset_name][model_name]
    config_url = catalog.CONFIG_CATALOG[dataset_name][model_name]

    # override folder destination:
    if 'model' not in os.listdir():
        os.mkdir('model')

    config_file_path, model_file_path = None, None

    for url in [model_url, config_url]:
        filename = url.split('/')[-1].split('?')[0]
        save_to_path = f"model/" + filename
        if 'config' in filename:
            config_file_path = copy.deepcopy(save_to_path)
        if 'model_final' in filename:
            model_file_path = copy.deepcopy(save_to_path)

        # skip if file exist in path
        if filename in os.listdir("model"):
            continue
        # Download file from URL
        r = requests.get(url, stream=True, headers={'user-agent': 'Wget/1.16 (linux-gnu)'})

        with open(save_to_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=4096):
                if chunk:
                    f.write(chunk)

    # load the label map
    label_map = catalog.LABEL_MAP_CATALOG[dataset_name]

    return lp.models.Detectron2LayoutModel(
        config_path=config_file_path,
        model_path=model_file_path,
        label_map=label_map
    )


In [ ]:
# Load the image of the PDF page (or convert PDF to image as above)
# image = Image.open("page_image.png")
pages = convert_from_path("News_Article__Fayetteville_Observer_The_NC___October_8_2024__p1.pdf")

# Error occurs: having trouble finding the model
# layout = lp.models.Detectron2LayoutModel("lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config")
# layout = lp.models.Detectron2LayoutModel("lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config")
# layout = lp.Detectron2LayoutModel(
# "./config.yaml",
# "./model_final.pth",
# extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
# label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"},
# )

# This works
# layout = load_model('lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config')


detected_layout = layout.detect(pages[0])

for block in detected_layout:
    segment = block.extract(pages)
    text = pytesseract.image_to_string(segment)  # OCR on each segment
    print(text)